In [49]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils import *
import resnet50
import pandas as pd
import numpy as np
from glob import glob
from keras.layers import Dense, Conv2D, BatchNormalization, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.metrics import log_loss
%matplotlib inline
cwd = os.getcwd()
input_dir  = os.path.join(os.path.dirname(cwd), 'data', 'crops_for_class', 'wrap')
batch_size=64
num_class = 447
img_shape=(3, 550, 550)
target_size = (550,550)

# Load Data

In [2]:
tr_data = load_array('tr_crop_550_conv_feats.bc')
val_data = load_array('val_crop_550_conv_feats.bc')
tr_c = load_array('tr_classes_550_crop.bc')
val_c = load_array('val_classes_550_crop.bc')

In [3]:
tr_data.shape

(3864, 512, 17, 17)

In [4]:
val_data.shape

(680, 512, 17, 17)

In [5]:
tr_c.shape

(3864,)

In [6]:
val_c.shape

(680,)

# Fit Model

In [34]:
def getlm():
    return Sequential([
            Flatten(input_shape=(512, 17, 17)),
            BatchNormalization(),
            Dense(512, activation='relu'),
            
            BatchNormalization(),
            Dense(num_class, activation='softmax')
        ])

In [35]:
model = getlm()

In [36]:
model.compile(Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [37]:
model.fit(tr_data, tr_c, validation_data=(val_data, val_c), shuffle=True, nb_epoch=3)

Train on 3864 samples, validate on 680 samples
Epoch 1/3
3864/3864 [==============================] - 10s - loss: 6.1623 - acc: 0.0380 - val_loss: 6.8270 - val_acc: 0.0676
Epoch 2/3
3864/3864 [==============================] - 10s - loss: 0.1995 - acc: 0.9990 - val_loss: 5.6785 - val_acc: 0.0721
Epoch 3/3
3864/3864 [==============================] - 10s - loss: 0.0237 - acc: 1.0000 - val_loss: 5.6219 - val_acc: 0.0794


In [38]:
model.save_weights('crop_class_5.61.h5')

In [39]:
model.optimizer.lr = 0.0001

In [40]:
model.fit(tr_data, tr_c, validation_data=(val_data, val_c), shuffle=True, nb_epoch=3)

Train on 3864 samples, validate on 680 samples
Epoch 1/3
3864/3864 [==============================] - 10s - loss: 0.0132 - acc: 1.0000 - val_loss: 5.6054 - val_acc: 0.0809
Epoch 2/3
3864/3864 [==============================] - 10s - loss: 0.0086 - acc: 1.0000 - val_loss: 5.5993 - val_acc: 0.0868
Epoch 3/3
3864/3864 [==============================] - 10s - loss: 0.0066 - acc: 1.0000 - val_loss: 5.5985 - val_acc: 0.0868


In [41]:
model.save_weights('crop_class_5.59.h5')

# Make Predictions

In [42]:
test_data = load_array('test_crop_550_conv_feats.bc')

In [44]:
test_preds = model.predict(test_data)

In [45]:
test_preds.shape

(6925, 447)

In [56]:
test_b = ImageDataGenerator().flow_from_directory(input_dir+'/test', shuffle=False, batch_size=1)

Found 6925 images belonging to 1 classes.


In [57]:
test_files = test_b.filenames

In [59]:
test_files = [f.split('/')[-1] for f in test_files]

In [64]:
class_ind = load_array('tr_indices_550_crop.bc')

In [70]:
from create_whale_sub import create_sub

In [71]:
sub = create_sub(test_preds, input_dir+'/train', input_dir+'/test')

Found 3864 images belonging to 447 classes.
Found 6925 images belonging to 1 classes.


In [72]:
sub.head()

,Image,whale_00195,whale_00442,whale_02411,whale_02608,whale_02839,whale_03103,whale_03227,whale_03623,whale_03728,...,whale_98618,whale_98633,whale_98645,whale_98746,whale_98939,whale_98996,whale_99243,whale_99326,whale_99558,whale_99573
0,w_8037.jpg,0.00045050677727,0.000869674724527,0.00825662724674,0.00118368084077,0.000973085174337,0.000943258753978,0.00186946091708,0.000869923969731,0.00179296627175,...,0.000636930402834,0.00115494965576,0.000742861826438,0.00362458219752,0.00273776077665,0.00415017595515,0.00268598506227,0.00234726583585,0.000823813665193,0.00102987687569
1,w_4494.jpg,0.0014414510224,0.000386572151911,0.0037086638622,0.00491609843448,0.0012361262925,0.00106194533873,0.0039384663105,0.00146460405085,0.000998414587229,...,0.00124140433036,0.00191066693515,0.000475915789139,0.00273562828079,0.000975822738837,0.00668139336631,0.00179154681973,0.00628502946347,0.00139774719719,0.0017936239019
2,w_4673.jpg,0.00164186931215,0.00405753217638,0.000823369191494,0.00137424224522,0.00167955853976,0.00197377311997,0.00181203987449,0.00153286242858,0.00445810426027,...,0.00360396131873,0.00366715388373,0.000745384721085,0.00399981811643,0.000541688932572,0.00125695951283,0.00797817669809,0.00184187258128,0.00616874173284,0.0020067980513
3,w_8273.jpg,0.000358745164704,0.000321814382914,0.00854142196476,0.0128783313558,0.000678644399159,0.00198532803915,0.000341606908478,0.000944130180869,0.000647524720989,...,0.0007751234225,0.000696532195434,0.00151734566316,0.00276913889684,0.00297112087719,0.00658826204017,0.000890566094313,0.000389042805182,0.00123505829833,0.00124370516278
4,w_7611.jpg,0.00216307747178,0.00210913992487,0.00114769500215,0.000811097736005,0.00210286863148,0.000820437737275,0.0020252126269,0.0021647720132,0.00148711644579,...,0.000955023861025,0.00156665907707,0.000539564003702,0.00937144272029,0.000224342438742,0.00198740744963,0.00362000684254,0.000952210451942,0.00274119595997,0.0024198780302


In [73]:
sub.to_csv('sub_crops_550.csv', index=False)
from IPython.display import FileLink
FileLink('sub_crops_550.csv')

/home/ubuntu/courses/deeplearning1/nbs/whale_recognition/sub_crops_550.csv